In [ ]:
!pip install "numpy<2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611310 sha256=49dd3d721cd23145d0e5da074aaa48c6baf530eeb29570edd372f2f05bf7b2fd
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [4]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
import numpy as np
from sklearn import model_selection as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, f1_score
from math import sqrt
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from collections import defaultdict

In [7]:
df = pd.read_csv('sample_data/kz.csv')
df.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [9]:
# Переименуем колонки для удобства (адаптируем под структуру нашего кода)
df = df.rename(columns={'user_id': 'userId', 'product_id': 'movieId', 'event_time': 'timestamp'})


In [10]:

df = df.drop_duplicates(subset=['order_id', 'movieId'])
df = df.dropna(subset=['userId', 'movieId'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [11]:
# Определяем k на основе медианного числа товаров повторного клиента
repeat_customers = df.groupby('userId').size()
k = int(repeat_customers.median())
k = max(3, min(k, 10))  # Ограничиваем k от 3 до 10
print(f'Selected k: {k}')

# Разделяем данные по дате (2020-09-01)
train_data = df[df['timestamp'] < '2020-09-01']
test_data = df[df['timestamp'] >= '2020-09-01']

def naive_popularity_based(train_data, k):
    popular_items = train_data['movieId'].value_counts().head(k).index.tolist()
    return popular_items

# Наивный алгоритм 2: Любимый товар клиента + (k-1) самых популярных
def naive_hybrid_based(train_data, k):
    # Самые популярные товары
    popular_items = train_data['movieId'].value_counts().head(k).index.tolist()

    # Любимый товар каждого клиента (самый частый)
    user_favorites = train_data.groupby('userId')['movieId'].apply(
        lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else popular_items[0]
    ).to_dict()

    recommendations = {}
    for user in set(train_data['userId'].unique()) | set(test_data['userId'].unique()):
        if user in user_favorites:
            favorite = user_favorites[user]
            # Убедимся, что favorite не входит в popular_items
            user_rec = [favorite] + [item for item in popular_items if item != favorite][:k-1]
        else:
            # Если пользователя нет в тренировочных данных, используем популярные товары
            user_rec = popular_items[:k]
        recommendations[user] = user_rec[:k]

    return recommendations

Selected k: 3


In [12]:
def evaluate_recommendations(recommendations, test_data, k):
    precisions = []
    recalls = []
    f1_scores = []

    # Группируем тестовые данные по пользователям
    test_user_items = test_data.groupby('userId')['movieId'].apply(set).to_dict()

    for user, actual_items in test_user_items.items():
        if user in recommendations:
            recommended_items = set(recommendations[user][:k])
        else:
            # Если пользователя нет в рекомендациях, пропускаем
            continue

        if len(recommended_items) == 0:
            precision = recall = f1 = 0
        else:
            # Вычисляем precision и recall
            true_positives = len(recommended_items & actual_items)
            precision = true_positives / len(recommended_items)
            recall = true_positives / len(actual_items) if len(actual_items) > 0 else 0

            # Вычисляем F1-score
            if precision + recall > 0:
                f1 = 2 * (precision * recall) / (precision + recall)
            else:
                f1 = 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Усредняем метрики по всем пользователям
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    avg_f1 = np.mean(f1_scores) if f1_scores else 0

    return avg_precision, avg_recall, avg_f1


In [13]:
popular_items = naive_popularity_based(train_data, k)
popular_recommendations = {user: popular_items for user in test_data['userId'].unique()}
hybrid_recommendations = naive_hybrid_based(train_data, k)
# hybrid_recommendations = naive_hybrid_based(train_data, k)
# random_recommendations = naive_random_based(train_data, k)

precision_pop, recall_pop, f1_pop  = evaluate_recommendations(popular_recommendations, test_data, 3)
# f1_hybrid = evaluate_naive_method(hybrid_recommendations, test_data)
# f1_random = evaluate_naive_method(random_recommendations, test_data)
precision_hyb, recall_hyb, f1_hyb = evaluate_recommendations(hybrid_recommendations, test_data, k)
print(f'Popularity Based - Precision: {precision_pop:.4f}, Recall: {recall_pop:.4f}, F1: {f1_pop:.4f}')
print(f'Hybrid Based - Precision: {precision_hyb:.4f}, Recall: {recall_hyb:.4f}, F1: {f1_hyb:.4f}')
# print(f'F1-Score Hybrid Based: {f1_hybrid:.4f}')
# print(f'F1-Score Random Based: {f1_random:.4f}')

Popularity Based - Precision: 0.0066, Recall: 0.0068, F1: 0.0052
Hybrid Based - Precision: 0.0114, Recall: 0.0155, F1: 0.0107


<br1>Продвинутые методы</br1>

In [16]:
df = pd.read_csv('sample_data/kz.csv')
df["event_time"] = pd.to_datetime(df["event_time"])
df["user_id"] = df["user_id"].astype(str)
df["product_id"] = df["product_id"].astype(str)


In [17]:
df["rating"] = 1.0

# -----------------------
# 3. Подготавливаем DataFrame для surprise
# -----------------------
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[["user_id", "product_id", "rating"]], reader)

In [18]:
trainset, testset = train_test_split(data, test_size=0.2, shuffle=False)  # последовательный сплит

In [19]:
algo = SVD(n_factors=20, random_state=42)
algo.fit(trainset)

# -----------------------
# 6. Предсказания на тесте
# -----------------------
predictions = algo.test(testset)

In [20]:
rmse = accuracy.rmse(predictions, verbose=False)
mae  = accuracy.mae(predictions, verbose=False)
print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}")


RMSE: 0.0067, MAE: 0.0014


In [21]:
def get_top_k(predictions, k=5):
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))
    # сортируем по оценке и оставляем топ-K
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k[uid] = [iid for iid, _ in user_ratings[:k]]
    return top_k

K = 5
top_k_recs = get_top_k(predictions, k=K)


In [22]:
precision_list = []
recall_list = []
f1_list = []

# формируем истинные покупки
true_user_items = df.groupby("user_id")["product_id"].apply(set).to_dict()

for uid, rec_items in top_k_recs.items():
    true_items = true_user_items.get(uid, set())
    if not true_items:
        continue
    hits = len(set(rec_items) & true_items)
    precision = hits / K
    recall = hits / len(true_items)
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

print(f"Средний Precision@{K}: {sum(precision_list)/len(precision_list):.4f}")
print(f"Средний Recall@{K}: {sum(recall_list)/len(recall_list):.4f}")
print(f"Средний F1@{K}: {sum(f1_list)/len(f1_list):.4f}")

Средний Precision@5: 0.4227
Средний Recall@5: 0.6625
Средний F1@5: 0.4237
